## Dataset Link: https://www.kaggle.com/anmolkumar/github-bugs-prediction/download

In [2]:
!pip install BeautifulSoup4
!pip install pydot

     |████████████████████████████████| 115 kB 2.9 MB/s eta 0:00:01
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
import os
import re
from bs4 import BeautifulSoup
from collections import Counter,defaultdict
import gc
import string
from unidecode import unidecode
import warnings
import numpy as np
import pandas as pd
import matplotlib.patches as mpatches
import matplotlib

import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense,
from keras.optimizers import Adam
import keras.backend as k
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional,GRU
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import transformers
from tqdm.notebook import tqdm
from tokenizers import BertWordPieceTokenizer
from transformers import AutoTokenizer, pipeline, TFDistilBertModel,TFAutoModel
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model

warnings.filterwarnings("ignore")
%matplotlib inline

In [4]:
''' reading files '''
train = pd.read_json("/kaggle/input/github-bugs-prediction/embold_train.json").reset_index(drop=True)
test = pd.read_json("/kaggle/input/github-bugs-prediction/embold_test.json").reset_index(drop=True)
train_extra= pd.read_json("../input/github-bugs-prediction/embold_train_extra.json").reset_index(drop=True)

In [5]:
def func(x):
    return x['title'] + " " + x['body']   

train['text'] = train.apply(lambda x : func(x),axis=1)
test['text'] = test.apply(lambda x : func(x),axis=1)

In [6]:
''' creating dictionary '''
dictn = {
        "i'm": "i am",
        "you're": "you are",
        "it's": "it is",
        "we're": "we are",
        "we'll": "we will",
        "That's":"that is",
        "haven't":"have not",
        "let's":"let us",
        "ain't": "am not / are not / is not / has not / have not",
        "aren't": "are not / am not",
        "can't": "cannot",
        "can't've": "cannot have",
        "'cause": "because",
        "could've": "could have",
        "couldn't": "could not",
        "couldn't've": "could not have",
        "didn't": "did not",
        "doesn't": "does not",
        "don't": "do not",
        "hadn't": "had not",
        "hadn't've": "had not have",
        "hasn't": "has not",
        "haven't": "have not",
        "he'd": "he had / he would",
        "he'd've": "he would have",
        "he'll": "he shall / he will",
        "he'll've": "he shall have / he will have",
        "he's": "he has / he is",
        "how'd": "how did",
        "how'd'y": "how do you",
        "how'll": "how will",
        "how's": "how has / how is / how does",
        "I'd": "I had / I would",
        "I'd've": "I would have",
        "I'll": "I shall / I will",
        "I'll've": "I shall have / I will have",
        "I'm": "I am",
        "I've": "I have",
        "isn't": "is not",
        "it'd": "it had / it would",
        "it'd've": "it would have",
        "it'll": "it shall / it will",
        "it'll've": "it shall have / it will have",
        "it's": "it has / it is",
        "let's": "let us",
        "ma'am": "madam",
        "mayn't": "may not",
        "might've": "might have",
        "mightn't": "might not",
        "mightn't've": "might not have",
        "must've": "must have",
        "mustn't": "must not",
        "mustn't've": "must not have",
        "needn't": "need not",
        "needn't've": "need not have",
        "o'clock": "of the clock",
        "oughtn't": "ought not",
        "oughtn't've": "ought not have",
        "shan't": "shall not",
        "sha'n't": "shall not",
        "shan't've": "shall not have",
        "she'd": "she had / she would",
        "she'd've": "she would have",
        "she'll": "she shall / she will",
        "she'll've": "she shall have / she will have",
        "she's": "she has / she is",
        "should've": "should have",
        "shouldn't": "should not",
        "shouldn't've": "should not have",
        "so've": "so have",
        "so's": "so as / so is",
        "that'd": "that would / that had",
        "that'd've": "that would have",
        "that's": "that has / that is",
        "there'd": "there had / there would",
        "there'd've": "there would have",
        "there's": "there has / there is",
        "they'd": "they had / they would",
        "they'd've": "they would have",
        "they'll": "they shall / they will",
        "they'll've": "they shall have / they will have",
        "they're": "they are",
        "they've": "they have",
        "to've": "to have",
        "wasn't": "was not",
        "we'd": "we had / we would",
        "we'd've": "we would have",
        "we'll": "we will",
        "we'll've": "we will have",
        "we're": "we are",
        "we've": "we have",
        "weren't": "were not",
        "what'll": "what shall / what will",
        "what'll've": "what shall have / what will have",
        "what're": "what are",
        "what's": "what has / what is",
        "what've": "what have",
        "when's": "when has / when is",
        "when've": "when have",
        "where'd": "where did",
        "where's": "where has / where is",
        "where've": "where have",
        "who'll": "who shall / who will",
        "who'll've": "who shall have / who will have",
        "who's": "who has / who is",
        "who've": "who have",
        "why's": "why has / why is",
        "why've": "why have",
        "will've": "will have",
        "won't": "will not",
        "won't've": "will not have",
        "would've": "would have",
        "wouldn't": "would not",
        "wouldn't've": "would not have",
        "y'all": "you all",
        "y'all'd": "you all would",
        "y'all'd've": "you all would have",
        "y'all're": "you all are",
        "y'all've": "you all have",
        "you'd": "you had / you would",
        "you'd've": "you would have",
        "you'll": "you shall / you will",
        "you'll've": "you shall have / you will have",
        "you're": "you are",
        "you've": "you have"
   }

In [7]:
''' regex '''
c_re = re.compile('(%s)' % '|'.join(dictn.keys()))

def expand_contractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [8]:
def removeEmoji(s):
        emoji_p = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        return emoji_p.sub(r'', s)

In [9]:
def removePunctuations(data):
    p_tag = re.compile(r'[^\w\s]')
    d = p_tag.sub(r'',data)
    return d

In [10]:
def remove_special_chars(d):
    '''
    Removes special characters which are specifically found in tweets.
    '''
    
    ''' Creating HTML tags '''
    beau_soup = BeautifulSoup(d, "html.parser")
    d = beau_soup.get_text()

    '''Convert www.* or https?://* to empty strings'''
    d = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','',d)

    '''Convert @username to empty strings'''
    d = re.sub('@[^\s]+','',d)
    
    ''' remove org.apache. like texts'''
    d =  re.sub('(\w+\.){2,}','',d)

    '''Remove additional white spaces'''
    d = re.sub('[\s]+', ' ', d)
    
    d = re.sub('\.(?!$)', '', d)

    '''Replace #word with word'''
    d = re.sub(r'#([^\s]+)', r'\1', d)

    return d 

In [11]:
def removeNonenglishCharac(s):
    return re.sub('\W+','', s )

In [12]:
extra_punc = ['','.', '``', '...', '\'s', '--', '-', 'n\'t', '_', '–','&']

sw_list = stopwords.words('english') + list(string.punctuation)+ extra_punc + ['u','the','us','say','that',
                     'he','me','she','get','rt','it','mt','via','not','and','let','so','say','dont','use','you']

In [13]:
def cleanText(d):
    ''' WordNetLemmatizer'''
    wordnet_lemmatizer = WordNetLemmatizer()
    
    ''' Stemming '''
    stem = Porterstem() 
    tokenizer=TweetTokenizer()
    d = unidecode(d)
    d = expand_contractions(d)
    tokens = tokenizer.tokenize(d)
    d = ' '.join([tok for tok in tokens if len(tok) > 2 if tok not in sw_list and not tok.isdigit()])
    d = re.sub('\b\w{,2}\b', '', d)
    d = re.sub(' +', ' ', d)
    
    ''' calling functions '''
    d = remove_special_chars(d)
    d = removeEmoji(d)
    d= [stem.stem(w) for w in d.split()]
    return ' '.join([wordnet_lemmatizer.lemmatize(word) for word in d])

In [14]:
train['text'] = train['text'].apply(lambda x: cleanText(x))

In [15]:
y_train = train_df['label']

''' train val split '''
X_train, X_val, y_train, y_val = train_test_split(train['text'], y_train, test_size=0.2, random_state=50)

In [16]:
try:    
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    print('TPU is not initialized ')
    tpu = None

Running on TPU  grpc://10.0.0.2:8470


In [17]:
if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()
    
print("Replicas num: ", strategy.num_replicas_in_sync)
AUTO = tf.data.experimental.AUTOTUNE

'''Configuration of hyperparameters'''
epochs = 3
bath_size = 16 * strategy.num_replicas_in_sync
max_len = 192

Replicas num:  8


In [18]:
'''Tokenize the data and separate them in chunks of 256 units'''

max_len=512
chunk_size=256

def quickEnc(t,token, maxlen=max_len):
    encoder_di = token.batch_encode_plus(t, return_attention_mask=False, return_token_type_ids=False, 
                                         pad_to_max_length=True, max_length=max_len, truncation=True)
    
    ''' converting into array '''
    return np.array(encoder_di["input_ids"])

In [19]:
''' Model function '''

def Model(transformer, X_train, X_val, batch_size, img_name, max_len=512):
    
    ''' input layer '''
    inp_w = Input(shape =(max_len,),dtype = tf.int32,name="input_word_ids")
    seq_out = transformer(inp_w)[0]
    cls_t = seq_out[:,0,:]
    output =  Dense(3,activation='softmax')(cls_t)
    
    ''' Model '''
    model = Model(inputs =inp_words_ids,outputs=output)
    
    ''' compile the model '''
    model.compile(Adam(lr=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    
    ''' lets see how model looks like '''
    model.summary()
    
    plot_model(model,to_file=img_name, show_shapes=True, show_layer_names=True, rankdir="TB", expand_nested=False, 
               dpi=96)
    
    ''' training '''
    model.fit(X_train, steps_per_epoch=batch_size, validation_data=X_val, epochs=epochs)

In [20]:
def delObects(*args):
    for arg in args:
        del arg
        gc.collect()

In [ ]:
''' BERT '''

with strategy.scope():

    dist_token = transformers.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
    X_train_enc= quickEnc(X_train.astype(str),dist_token, maxlen=max_len)
    X_val_ebc = quickEnc(X_val.astype(str), dist_token, maxlen=max_len)
    
    ''' input pipe line '''
    train_ds = (
        tf.data.Dataset.from_tensor_slices((X_train_enc, y_train))
        .repeat()
        .shuffle(2048)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )
    
    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val_ebc, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)                
    )
    
    transf_layer = TFAutoModel.from_pretrained('distilbert-base-multilingual-cased')
    
    ''' Model '''
    Model(transf_layer, train_ds, val_ds, X_train_enc.shape[0], "Distilbert_Multilingual_Transformer.png", 
          max_len=max_len)
    
    delObects(X_train_enc, X_val_enc, train_ds, val_ds, dist_token, transf_layer)

Downloading:   0%|          | 0.00/996k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/911M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-multilingual-cased were not used when initializing TFDistilBertModel: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 192)]             0         
_________________________________________________________________
tf_distil_bert_model (TFDist TFBaseModelOutput(last_hi 134734080 
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 768)               0         
_________________________________________________________________
dense (Dense)                (None, 3)                 2307      
Total params: 134,736,387
Trainable params: 134,736,387
Non-trainable params: 0
_________________________________________________________________
Epoch 1/3
120000/120000 [==============================] - 9010s 75ms/step - loss: 0.1737 - accuracy: 0.9333 - val_loss: 2.2630 - val_accuracy: 0.7676
Epoch 2/3
   589/120000 [..............................] - ETA: 2:29:19 - loss: 0.0049 - accur

In [ ]:
''' Albert Transformer '''

with strategy.scope():

    alb_token = transformers.AlbertTokenizer.from_pretrained('albert-base-v1')
    X_train_enc= quickEnc(X_train.astype(str), alb_token, maxlen=max_len)
    X_val_enc = quickEnc(X_val.astype(str), alb_token, maxlen=max_len)
    
    ''' input pipe line '''
    train_ds = (
                    tf.data.Dataset.from_tensor_slices((X_train_enc, y_train)
                    .repeat()
                    .shuffle(2048)
                    .batch(BATCH_SIZE)
                    .prefetch(AUTO)
                    )
    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val_enc, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)                
    )
        
    transf_layer = TFAutoModel.from_pretrained('albert-base-v1')
    
    Model(transf_layer, train_ds, val_ds, X_train_enc.shape[0], "AlbertTransformer.png", max_len=max+len)
    
    delObects(X_train_enc, X_val_enc, train_ds, val_ds, alb_token, transf_layer)

In [ ]:
''' XLM Roberta/Roberta '''

with strategy.scope():

    xlm_roberta_token = transformers.XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
    X_train_enc = quickEnc(X_train.astype(str), xlm_roberta_token, maxlen=max_len)
    X_val_enc = quickEnc(X_val.astype(str), xlm_roberta_token, maxlen=max_len)
    
    ''' input pipe line '''
    train_ds = (
                tf.data.Dataset.from_tensor_slices((X_train_enc, y_train))
                .repeat()
                .shuffle(2048)
                .batch(BATCH_SIZE)
                .prefetch(AUTO)
                )
    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val_enc, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)                
    )
    
    transf_layer = TFAutoModel.from_pretrained('xlm-roberta-base')
    
    Model(transf_layer, train_ds, val_ds, X_train_enc.shape[0], "Roberta-Transformer.png",max_len=max_len)
    
    delObects(X_train_enc, X_val_enc, train_ds, val_ds, xlm_roberta_token, transf_layer)

In [ ]:
''' GPT-Generative Pretraining '''

with strategy.scope():

    gpt2_token = transformers.GPT2Tokenizer.from_pretrained('gpt2-medium')
    X_train_enc= quickEnc(X_train.astype(str),gpt2_token, maxlen=max_len)
    X_val_enc = quickEnc(X_val.astype(str), gpt2_token, maxlen=max_len)
    
    ''' input pipe line '''
    train_ds = (
                tf.data.Dataset.from_tensor_slices((X_train_enc, y_train))
                .repeat()
                .shuffle(2048)
                .batch(BATCH_SIZE)
                .prefetch(AUTO)
                )
    val_ds = (
        tf.data.Dataset.from_tensor_slices((X_val_enc, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)                
    )
    
    transf_layer = TFAutoModel.from_pretrained('gpt2-medium')
    
    Model(transf_layer, train_ds, val_ds, X_train_enc.shape[0], "GPT2-Transformer.png", max_len=max_len)
    
    delObects(X_train_enc, X_val_enc, train_ds, val_ds, gpt2_token,transf_layer)